# Doğal Dil İşleme Dersi - Ödev 1: Vektörleştirme (Vectorization)

Bu notebook, ön işleme adımlarından geçirilmiş metin verilerinin TF-IDF ve Word2Vec yöntemleriyle vektörleştirilmesini içermektedir.

## 1. Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import time
from tqdm import tqdm

# Grafik ayarları
plt.style.use('ggplot')
sns.set(font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)

## 2. İşlenmiş Verilerin Yüklenmesi

In [ ]:
# İşlenmiş verileri yükleme
processed_data_dir = "../data/processed/"
models_dir = "../models/"

# Dizinlerin var olduğundan emin olalım
os.makedirs(models_dir, exist_ok=True)

# Stemming ve lemmatization sonucu elde edilen verileri yükleme
'''
stemmed_df = pd.read_csv(os.path.join(processed_data_dir, "stemmed_data.csv"))
lemmatized_df = pd.read_csv(os.path.join(processed_data_dir, "lemmatized_data.csv"))

print(f"Stemming sonrası veri boyutu: {stemmed_df.shape}")
print(f"Lemmatization sonrası veri boyutu: {lemmatized_df.shape}")
'''

## 3. TF-IDF Vektörleştirme

TF-IDF (Term Frequency-Inverse Document Frequency), bir kelimenin bir belgede ne kadar önemli olduğunu ölçen istatistiksel bir yöntemdir. Bu bölümde, hem stemming hem de lemmatization sonucu elde edilen veriler için TF-IDF vektörleştirme işlemini gerçekleştireceğiz.

In [ ]:
# Stemming sonrası veriler için TF-IDF vektörleştirme
'''
# TF-IDF vektörleştirici oluşturma
tfidf_vectorizer_stemmed = TfidfVectorizer(max_features=5000)  # En sık kullanılan 5000 kelimeyi kullan

# Vektörleştirme işlemini gerçekleştirme
tfidf_matrix_stemmed = tfidf_vectorizer_stemmed.fit_transform(stemmed_df['text'])

# Özellikleri (kelimeleri) alma
feature_names_stemmed = tfidf_vectorizer_stemmed.get_feature_names_out()

# TF-IDF matrisini DataFrame'e dönüştürme
tfidf_df_stemmed = pd.DataFrame(tfidf_matrix_stemmed.toarray(), columns=feature_names_stemmed)
tfidf_df_stemmed.index = stemmed_df['document_id']

# TF-IDF DataFrame'ini kaydetme
tfidf_df_stemmed.to_csv(os.path.join(processed_data_dir, "tfidf_stemmed.csv"))

print(f"Stemming sonrası TF-IDF matris boyutu: {tfidf_df_stemmed.shape}")
'''

In [ ]:
# Lemmatization sonrası veriler için TF-IDF vektörleştirme
'''
# TF-IDF vektörleştirici oluşturma
tfidf_vectorizer_lemmatized = TfidfVectorizer(max_features=5000)  # En sık kullanılan 5000 kelimeyi kullan

# Vektörleştirme işlemini gerçekleştirme
tfidf_matrix_lemmatized = tfidf_vectorizer_lemmatized.fit_transform(lemmatized_df['text'])

# Özellikleri (kelimeleri) alma
feature_names_lemmatized = tfidf_vectorizer_lemmatized.get_feature_names_out()

# TF-IDF matrisini DataFrame'e dönüştürme
tfidf_df_lemmatized = pd.DataFrame(tfidf_matrix_lemmatized.toarray(), columns=feature_names_lemmatized)
tfidf_df_lemmatized.index = lemmatized_df['document_id']

# TF-IDF DataFrame'ini kaydetme
tfidf_df_lemmatized.to_csv(os.path.join(processed_data_dir, "tfidf_lemmatized.csv"))

print(f"Lemmatization sonrası TF-IDF matris boyutu: {tfidf_df_lemmatized.shape}")
'''

## 4. TF-IDF Sonuçlarının İncelenmesi

In [ ]:
# Stemming sonrası TF-IDF matrisinden ilk 5 satır ve 10 sütunu gösterme
'''
print("Stemming sonrası TF-IDF matrisinden örnek:")
tfidf_df_stemmed.iloc[:5, :10]
'''

In [ ]:
# Lemmatization sonrası TF-IDF matrisinden ilk 5 satır ve 10 sütunu gösterme
'''
print("Lemmatization sonrası TF-IDF matrisinden örnek:")
tfidf_df_lemmatized.iloc[:5, :10]
'''

In [ ]:
# Stemming sonrası en yüksek TF-IDF değerine sahip kelimeleri gösterme
'''
# Her belge için en yüksek TF-IDF değerine sahip 5 kelimeyi bulma
def get_top_tfidf_words(tfidf_df, n=5):
    results = []
    for i in range(len(tfidf_df)):
        row = tfidf_df.iloc[i]
        top_indices = row.nlargest(n).index
        top_values = row.nlargest(n).values
        results.append({
            'document_id': tfidf_df.index[i],
            'top_words': [(word, value) for word, value in zip(top_indices, top_values)]
        })
    return results

top_stemmed_words = get_top_tfidf_words(tfidf_df_stemmed)

# İlk 5 belge için en önemli kelimeleri gösterme
for i, result in enumerate(top_stemmed_words[:5]):
    print(f"Belge {result['document_id']} için en önemli kelimeler:")
    for word, value in result['top_words']:
        print(f"  {word}: {value:.4f}")
    print()
'''

In [ ]:
# Lemmatization sonrası en yüksek TF-IDF değerine sahip kelimeleri gösterme
'''
top_lemmatized_words = get_top_tfidf_words(tfidf_df_lemmatized)

# İlk 5 belge için en önemli kelimeleri gösterme
for i, result in enumerate(top_lemmatized_words[:5]):
    print(f"Belge {result['document_id']} için en önemli kelimeler:")
    for word, value in result['top_words']:
        print(f"  {word}: {value:.4f}")
    print()
'''

## 5. Word2Vec Vektörleştirme

Word2Vec, kelimeleri anlamsal olarak benzer vektörlere dönüştüren bir yöntemdir. Bu bölümde, hem stemming hem de lemmatization sonucu elde edilen veriler için farklı parametrelerle Word2Vec modelleri eğiteceğiz.

In [ ]:
# Word2Vec için verileri hazırlama
'''
# Stemming sonrası verileri kelime listelerine dönüştürme
stemmed_sentences = [text.split() for text in stemmed_df['text']]

# Lemmatization sonrası verileri kelime listelerine dönüştürme
lemmatized_sentences = [text.split() for text in lemmatized_df['text']]
'''

In [ ]:
# Word2Vec model parametreleri
parameters = [
    {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
    {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
    {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
    {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]

In [ ]:
# Word2Vec modellerini eğitme fonksiyonu
def train_word2vec_models(sentences, params_list, prefix):
    models_info = []
    
    for params in tqdm(params_list, desc=f"Training {prefix} models"):
        model_type = params['model_type']
        window = params['window']
        vector_size = params['vector_size']
        
        # Model adını oluşturma
        model_name = f"{prefix}_{model_type}_win{window}_dim{vector_size}"
        
        # Eğitim başlangıç zamanı
        start_time = time.time()
        
        # Word2Vec modelini oluşturma ve eğitme
        sg = 1 if model_type == 'skipgram' else 0  # sg=1: Skip-gram, sg=0: CBOW
        model = Word2Vec(sentences=sentences, vector_size=vector_size, window=window, 
                         min_count=2, workers=4, sg=sg)
        
        # Eğitim bitiş zamanı
        end_time = time.time()
        training_time = end_time - start_time
        
        # Modeli kaydetme
        model_path = os.path.join(models_dir, f"{model_name}.model")
        model.save(model_path)
        
        # Model bilgilerini kaydetme
        model_info = {
            'model_name': model_name,
            'model_type': model_type,
            'window': window,
            'vector_size': vector_size,
            'training_time': training_time,
            'model_size': os.path.getsize(model_path) / (1024*1024),  # MB cinsinden
            'vocabulary_size': len(model.wv.key_to_index),
            'model_path': model_path
        }
        
        models_info.append(model_info)
        
    return models_info

In [ ]:
# Stemming sonrası veriler için Word2Vec modellerini eğitme
'''
stemmed_models_info = train_word2vec_models(stemmed_sentences, parameters, "stemmed")
'''

In [ ]:
# Lemmatization sonrası veriler için Word2Vec modellerini eğitme
'''
lemmatized_models_info = train_word2vec_models(lemmatized_sentences, parameters, "lemmatized")
'''

## 6. Word2Vec Model Bilgilerinin Gösterilmesi

In [ ]:
# Stemming sonrası Word2Vec model bilgilerini gösterme
'''
stemmed_models_df = pd.DataFrame(stemmed_models_info)
print("Stemming sonrası Word2Vec modelleri:")
stemmed_models_df[['model_name', 'model_type', 'window', 'vector_size', 'training_time', 'model_size', 'vocabulary_size']]
'''

In [ ]:
# Lemmatization sonrası Word2Vec model bilgilerini gösterme
'''
lemmatized_models_df = pd.DataFrame(lemmatized_models_info)
print("Lemmatization sonrası Word2Vec modelleri:")
lemmatized_models_df[['model_name', 'model_type', 'window', 'vector_size', 'training_time', 'model_size', 'vocabulary_size']]
'''

## 7. Word2Vec Modellerinin Benzerlik Analizi

Bu bölümde, eğitilen Word2Vec modellerini kullanarak kelime benzerliklerini inceleyeceğiz.

In [ ]:
# Benzerlik analizi için örnek kelime seçme
'''
# Veri setinize göre önemli bir kelime seçin
example_word = "example"  # Veri setinize göre değiştirin
'''

In [ ]:
# Stemming sonrası modeller için benzerlik analizi
'''
print("Stemming sonrası modeller için benzerlik analizi:")
for model_info in stemmed_models_info:
    model_path = model_info['model_path']
    model = Word2Vec.load(model_path)
    
    print(f"\nModel: {model_info['model_name']}")
    
    try:
        # Örnek kelimeye en benzer 5 kelimeyi bulma
        similar_words = model.wv.most_similar(example_word, topn=5)
        print(f"'{example_word}' kelimesine en benzer 5 kelime:")
        for word, similarity in similar_words:
            print(f"  {word}: {similarity:.4f}")
    except KeyError:
        print(f"'{example_word}' kelimesi modelin kelime dağarcığında bulunmuyor.")
        # Alternatif bir kelime seçme
        try:
            # Modelin kelime dağarcığından rastgele bir kelime seçme
            alt_word = list(model.wv.key_to_index.keys())[0]
            similar_words = model.wv.most_similar(alt_word, topn=5)
            print(f"Alternatif olarak '{alt_word}' kelimesine en benzer 5 kelime:")
            for word, similarity in similar_words:
                print(f"  {word}: {similarity:.4f}")
        except:
            print("Benzerlik analizi yapılamadı.")
'''

In [ ]:
# Lemmatization sonrası modeller için benzerlik analizi
'''
print("Lemmatization sonrası modeller için benzerlik analizi:")
for model_info in lemmatized_models_info:
    model_path = model_info['model_path']
    model = Word2Vec.load(model_path)
    
    print(f"\nModel: {model_info['model_name']}")
    
    try:
        # Örnek kelimeye en benzer 5 kelimeyi bulma
        similar_words = model.wv.most_similar(example_word, topn=5)
        print(f"'{example_word}' kelimesine en benzer 5 kelime:")
        for word, similarity in similar_words:
            print(f"  {word}: {similarity:.4f}")
    except KeyError:
        print(f"'{example_word}' kelimesi modelin kelime dağarcığında bulunmuyor.")
        # Alternatif bir kelime seçme
        try:
            # Modelin kelime dağarcığından rastgele bir kelime seçme
            alt_word = list(model.wv.key_to_index.keys())[0]
            similar_words = model.wv.most_similar(alt_word, topn=5)
            print(f"Alternatif olarak '{alt_word}' kelimesine en benzer 5 kelime:")
            for word, similarity in similar_words:
                print(f"  {word}: {similarity:.4f}")
        except:
            print("Benzerlik analizi yapılamadı.")
'''

## 8. Model Başarısının Değerlendirilmesi

Bu bölümde, eğitilen modellerin başarısını değerlendireceğiz.

In [ ]:
# Model başarısının değerlendirilmesi
'''
print("Model Başarısının Değerlendirilmesi:")
print("\nStemming vs Lemmatization:")
print("- Stemming, kelimeleri köklerine indirgerken anlamsal bilgileri kaybedebilir.")
print("- Lemmatization, kelimeleri anlamsal köklerine indirgediği için daha anlamlı sonuçlar verebilir.")

print("\nCBOW vs Skip-gram:")
print("- CBOW, bir kelimenin bağlamından (çevresindeki kelimelerden) o kelimeyi tahmin etmeye çalışır.")
print("- Skip-gram, bir kelimeden o kelimenin bağlamını (çevresindeki kelimeleri) tahmin etmeye çalışır.")
print("- Skip-gram genellikle nadir kelimeler için daha iyi performans gösterir, ancak eğitim süresi daha uzundur.")

print("\nPencere Boyutu (Window Size):")
print("- Küçük pencere boyutu (2), yakın kelimelere daha fazla önem verir ve sözdizimsel ilişkileri daha iyi yakalar.")
print("- Büyük pencere boyutu (4), daha geniş bağlamı dikkate alır ve anlamsal ilişkileri daha iyi yakalar.")

print("\nVektör Boyutu (Vector Size):")
print("- Küçük vektör boyutu (100), eğitim süresini kısaltır ancak temsil kapasitesi sınırlıdır.")
print("- Büyük vektör boyutu (300), daha zengin temsiller sağlar ancak eğitim süresi daha uzundur ve aşırı öğrenme riski vardır.")

print("\nBeklenen En Başarılı Model:")
print("Lemmatization sonrası, Skip-gram, pencere boyutu 4 ve vektör boyutu 300 olan modelin en başarılı olması beklenir.")
print("Çünkü:")
print("- Lemmatization, anlamsal bilgileri koruduğu için daha iyi sonuçlar verir.")
print("- Skip-gram, nadir kelimeler için daha iyi performans gösterir.")
print("- Büyük pencere boyutu, daha geniş bağlamı dikkate alır.")
print("- Büyük vektör boyutu, daha zengin temsiller sağlar.")
'''

## 9. Sonuç

Bu notebook'ta, ön işleme adımlarından geçirilmiş metin verilerini TF-IDF ve Word2Vec yöntemleriyle vektörleştirdik. Her iki yöntem için de hem stemming hem de lemmatization sonucu elde edilen verileri kullandık. Word2Vec için farklı parametrelerle 16 farklı model eğittik ve bu modellerin performansını değerlendirdik.